# <center>Trabajo especial N° 1</center>

Lo que sigue está basado en el modelo presentado en el paper _Strategies for controlling the medical and socio-economic costs of
the Corona pandemic_, publicado en abril de 2020. Se trata de una adaptación del modelo SIR que incorpora al análisis los efectos de dos políticas de contención distintas.

A pesar de su simplicidad me pareció ingeniosa la manera de incorporar los efectos que tienen las distintas políticas sobre el ritmo de contagio. Esperaría que el poder de generalización sea bastante pobre, pienso por ejemplo en diferencias culturales. Las medidas gubernamentales implementadas seguramente tengan más peso en un lugar como China que en por ejemplo Francia. A pesar de que no esperaría que ningún país se ajuste a uno de estos dos escenarios (seguramente se trate de una combinación en la mayoría de los casos), pienso que sería útil como guía para ir corrigiendo o ajustando políticas a medida que se cursa la epidemia.

El análisis de los costos agrega complejidad al modelo pero a su vez lo hace de una manera muy simple, lo cual me parece le quita poder de generalización. Los costos de cada país seguramente dependan bastante por ejemplo del estado de desarrollo de su infraestructura de salud, entre otras cosas.

Abajo dejo los plots de los modelos que armé. En ambos casos agregué una manera de editar los parámetros de manera dinámica para poder comparar escenarios de manera inmediata. En el caso del análisis de los costos me parece que cometí algún error porque quise reproducir los ejemplos del paper y no he podido.

Se puede encontrar más detalles de implementación [aqui](https://gitlab.com/jouse/opti/-/tree/master/opti)

In [29]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interactive
DEFAULT_TOTAL_TIME = 26

In [30]:
def sir_model_simulator(
    infected_zero=0.001,
    g_zero=3,
    alpha=20,
    theta=0.1,
    reaction_policy='history_aware',
    coeff_k=0.3,
    coeff_m=0.25 * (2 / 52),
):

    infected = [infected_zero]
    removed = infected.copy()
    medical_costs = []
    dispo_costs = []

    if reaction_policy == "short_sighted":
        transmition_rate = g_zero / (1 + theta * alpha * infected[0])
    else:
        transmition_rate = g_zero / (1 + theta * alpha * removed[0])

    for t in range(1, DEFAULT_TOTAL_TIME):
        new_infections = transmition_rate * infected[t - 1] * (1 - removed[t - 1])
        infected.append(new_infections)
        removed.append(removed[t - 1] + new_infections)

        if new_infections > 10 ** (-5):
            medical_costs.append(coeff_k * infected[t])
            dispo_costs.append(coeff_m * (1 - transmition_rate / g_zero))

        if reaction_policy == "short_sighted":
            transmition_rate = g_zero / (1 + theta * alpha * infected[t])
        else:
            transmition_rate = g_zero / (1 + theta * alpha * removed[t])

    return {
        "infected": infected,
        "removed": removed,
        "dispo_costs": sum(dispo_costs),
        "medical_costs": sum(medical_costs),
    }

In [31]:
def plot_infected_vs_time(
    infected_zero=0.001,
    g_zero=3,
    alpha=20,
):
    result_ss = sir_model_simulator(
        infected_zero=infected_zero,
        g_zero=g_zero,
        alpha=alpha,
        reaction_policy='short_sighted'
    )
    result_ha = sir_model_simulator(
        infected_zero=infected_zero,
        g_zero=g_zero,
        alpha=alpha,
        reaction_policy='history_aware'
    )

    time_axis = np.arange(0, DEFAULT_TOTAL_TIME)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=time_axis, y=result_ss['infected'], mode='lines', name='infected s.s.')
    )
    fig.add_trace(
        go.Scatter(x=time_axis, y=result_ha['infected'], mode='lines', name='infected h.a.')
    )
    fig.show()


plot_infected = interactive(
    plot_infected_vs_time,
    infected_zero=widgets.FloatSlider(value=0.02, min=0.01, max=0.1, step=0.001),
    g_zero=widgets.FloatSlider(value=2.5, min=1, max=20, step=0.5),
    alpha=widgets.IntSlider(value=10, min=0, max=30, step=1),
)
display(plot_infected)

interactive(children=(FloatSlider(value=0.02, description='infected_zero', max=0.1, min=0.01, step=0.001), Flo…

In [25]:
def plot_costs_vs_alpha(coeff_k, coeff_m):
    alpha_axis = np.arange(0, 40)
    medic_costs_ss = []
    dispo_costs_ss = []
    medic_costs_ha = []
    dispo_costs_ha = []
    for a in alpha_axis:
        results_ss = sir_model_simulator(
            alpha=a, coeff_k=coeff_k, coeff_m=coeff_m, reaction_policy='short_sighted'
        )
        results_ha = sir_model_simulator(
            alpha=a, coeff_k=coeff_k, coeff_m=coeff_m, reaction_policy='history_aware'
        )

        medic_costs_ss.append(results_ss['medical_costs'])
        dispo_costs_ss.append(results_ss['dispo_costs'])
        medic_costs_ha.append(results_ha['medical_costs'])
        dispo_costs_ha.append(results_ha['dispo_costs'])
    
    fig = make_subplots(
        rows=3,
        cols=1,
        subplot_titles=("Total costs", "Social dist. costs", "Economic costs")
    )
    fig.add_trace(
        go.Scatter(
            x=alpha_axis,
            y=np.array(dispo_costs_ss) + np.array(medic_costs_ss),
            mode='lines',
            name='Short sighted',
            line=dict(color='firebrick'),
        ), row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=alpha_axis,
            y=np.array(dispo_costs_ha) + np.array(medic_costs_ha),
            mode='lines',
            name='History aware',
            line=dict(color='royalblue')
        ), row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=alpha_axis,
            y=np.array(dispo_costs_ss),
            showlegend=False,
            mode='lines',
            line=dict(color='firebrick'),
        ), row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=alpha_axis,
            y=np.array(dispo_costs_ha),
            mode='lines',
            showlegend=False,
            line=dict(color='royalblue')
        ), row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=alpha_axis,
            y=np.array(medic_costs_ss),
            mode='lines',
            showlegend=False,
            line=dict(color='firebrick'),
        ), row=3, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=alpha_axis,
            y=np.array(medic_costs_ha),
            mode='lines',
            showlegend=False,
            line=dict(color='royalblue')
        ), row=3, col=1
    )
    fig.show()

plot_costs = interactive(
    plot_costs_vs_alpha,
    coeff_k=widgets.FloatSlider(value=0.3, min=0.1, max=1, step=0.01),
    coeff_m=widgets.FloatSlider(value=0.25, min=0.1, max=1, step=0.01)
)
display(plot_costs)

interactive(children=(FloatSlider(value=0.3, description='coeff_k', max=1.0, min=0.1, step=0.01), FloatSlider(…